In [10]:
import pandas as pd 
import numpy as np

from keras.callbacks import EarlyStopping
from keras.models import Sequential
from keras.layers import Dense
from keras.datasets import mnist

from tqdm import tqdm
from keras.applications.vgg16 import preprocess_input

from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
%matplotlib inline

In [11]:
def add_padding(dataset=None, pad_width = 0, constant_values=0):
    padded_dataset = []
    
    for i in dataset:
        padded = np.pad(i, pad_width=pad_width, mode='constant', constant_values=constant_values)
        padded_dataset.append(padded[:, :, np.newaxis])
    
    return padded_dataset

# how to use the preprocess input ? 
def preprocess2(dataset, dimension):
    new_set = []
    
    for x in dataset:
        p = np.repeat(x, 3, axis=-1)
        p = preprocess_input(p)
        
        new_set.append(p)
    
    new_set = np.array(new_set)
    
    new_set = new_set.reshape(new_set.shape[0], dimension, -1)
    new_set = new_set.astype('float32')
        
    return new_set

def preprocess(dataset, dimension):
    new_dataset = dataset.reshape(dataset.shape[0], dimension)
    new_dataset = new_dataset.astype('float32')
        
    return new_dataset

def write_matrix_to_file(matrix=None, file_name="random.txt"):
    with open(file_name, 'wb') as f:
        for line in matrix:
            np.savetxt(f, line, delimiter=' ')

In [12]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

new_x_train = add_padding(x_train, pad_width=2, constant_values=0)
new_x_test = add_padding(x_test, pad_width=2, constant_values=0)

In [13]:
new_x_train = np.asarray(new_x_train)
new_x_test = np.asarray(new_x_test)

dim_data = np.prod(new_x_train.shape[1:])

new_x_train = preprocess(new_x_train, dim_data)
new_x_test = preprocess(new_x_test, dim_data)

In [ ]:
model = Sequential()

model.add(Dense(1024, input_shape=(dim_data,)))

model.add(Dense(100, activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(100, activation='relu'))

model.add(Dense(512, activation='relu'))

model.compile(loss='mse', 
              optimizer='adam', 
              metrics=['mse'])

earlyStopping = EarlyStopping(monitor='val_loss', 
                              patience=25)

history = model.fit(new_x_train,
                    y_train,
                    epochs=250, 
                    batch_size=1000, 
                    verbose=0, 
                    validation_split=0.2, 
                    callbacks=[earlyStopping])

In [ ]:
model.summary()

In [ ]:
plt.plot(history.history['loss'], label='train loss')
plt.plot(history.history['val_loss'], label='validation loss')
plt.legend()

In [ ]:
test_results = model.evaluate(new_x_train, y_train, verbose=1)

In [ ]:
train_features = model.predict(new_x_train)

In [ ]:
test_features = model.predict(new_x_test)

In [ ]:
train_features = np.matrix(train_features)
write_matrix_to_file(train_features, file_name="simple_ann_train_features.out")

In [ ]:
test_features = np.matrix(test_features)
write_matrix_to_file(test_features, file_name="simple_ann_test_features.out")